In [1]:
import streamlit as st
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Content, FinishReason, SafetySetting
import vertexai.preview.generative_models as generative_models
import os
import requests
import google.auth.transport.requests
from google.oauth2 import service_account
import google.oauth2.id_token
import vertexai

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\ASUS\Documents\Job\Datalabs\2025\service_account\dla-gen-ai-specialization-a03e18685e07.json" # Isi dengan Service Account

In [12]:
PROJECT_ID = "844021890758"
COLLECTION_NAME = "default_collection"
ENGINE_NAME = "demo2-test_1736323106527"

In [4]:
def search_with_answer(query,session,agent):
    url_search = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:search"
    # API credential
    credentials = service_account.Credentials.from_service_account_file(
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'],
        scopes=['https://www.googleapis.com/auth/cloud-platform']
    )
    request_token = google.auth.transport.requests.Request()
    credentials.refresh(request_token)
    headers = {
        "Authorization": f"Bearer {credentials.token}",
        "Content-Type": "application/json"
    }
    # ngelakuin search file dulu baru summary, ini buat data searchnya
    if session == "":
        session = f"projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/sessions/-"
    else:
        pass
    request_data_search = {
        "query": query,
        "pageSize": 10,
        "queryExpansionSpec": {
            "condition": "AUTO"
        },
        "spellCorrectionSpec": {
            "mode": "AUTO"
        },
        "contentSearchSpec": {
            "snippetSpec": {
                "returnSnippet": True
            },
            "extractiveContentSpec": {
                "maxExtractiveAnswerCount": 1
            }
        },
        "session": session
    }
    request_search = requests.post(url=url_search, json=request_data_search,headers=headers)
    search = request_search.json()
    # ini bagian api call untuk summary
    url_answer = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:answer"

    request_data_answer = {
        "query": {
            "text": query,
            "queryId": search['sessionInfo']['queryId']
        },
        "session": search['sessionInfo']['name'],
        "relatedQuestionsSpec": {
            "enable": True
        },
        "answerGenerationSpec": {
            "ignoreAdversarialQuery": True,
            "ignoreNonAnswerSeekingQuery": True,
            "ignoreLowRelevantContent": True,
            "includeCitations": True,
            "promptSpec": {
                "preamble": """Anda adalah agent chatbot internal untuk perusahaan WOM Finance Jawablah dalam bahasa indonesia jangan dengan bahasa inggris,untuk jawaban yang tidak ditemukan harap balas dengan "Jawaban tidak ditemukan, silahkan menghubungi tim OPP pada email OPP@wom.co.id. Terima Kasih" """
            },
            "modelSpec": {
                "modelVersion": "gemini-1.5-flash-001/answer_gen/v2"
            }
        }
    }

    request_answer = requests.post(url_answer, json=request_data_answer,headers=headers)
    answer = request_answer.json()

    return answer

In [24]:
query = "laptop reccomendations"
session = ''
agent = "demo2-test_1736323106527"

In [25]:
url_search = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:search"
# API credential
credentials = service_account.Credentials.from_service_account_file(
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'],
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)
request_token = google.auth.transport.requests.Request()
credentials.refresh(request_token)
headers = {
    "Authorization": f"Bearer {credentials.token}",
    "Content-Type": "application/json"
}
# ngelakuin search file dulu baru summary, ini buat data searchnya
if session == "":
    session = f"projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/sessions/-"
else:
    pass
request_data_search = {
    "query": query,
    "pageSize": 10,
    "queryExpansionSpec": {
        "condition": "AUTO"
    },
    "spellCorrectionSpec": {
        "mode": "AUTO"
    },
    "contentSearchSpec": {
        "snippetSpec": {
            "returnSnippet": True
        },
        "extractiveContentSpec": {
            "maxExtractiveAnswerCount": 1
        }
    },
    "session": session
}
request_search = requests.post(url=url_search, json=request_data_search,headers=headers)
search = request_search.json()

In [26]:
# ini bagian api call untuk summary
url_answer = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:answer"

request_data_answer = {
    "query": {
        "text": query,
        "queryId": search['sessionInfo']['queryId']
    },
    "session": search['sessionInfo']['name'],
    "relatedQuestionsSpec": {
        "enable": True
    },
    "answerGenerationSpec": {
        "ignoreAdversarialQuery": True,
        "ignoreNonAnswerSeekingQuery": True,
        "ignoreLowRelevantContent": True,
        "includeCitations": True,
        "promptSpec": {
            "preamble": """"""
        },
        "modelSpec": {
            "modelVersion": "gemini-1.5-flash-001/answer_gen/v2"
        }
    }
}

request_answer = requests.post(url_answer, json=request_data_answer,headers=headers)
answer = request_answer.json()

In [27]:
answer['answer']['answerText']

'A summary could not be generated for your search query. Here are some search results.'

In [39]:
search['results'][0]['document']['derivedStructData']['link']

'gs://gen-ai-specialization-demo2/data-hugging-face/text/output_7.txt'

In [36]:
for i in range(len(search['results'])):
    print(f"[{i}]{search['results'][i]['document']['derivedStructData']['extractive_answers'][0]['content']}")

[0]The ThinkPad X1 Fold is an innovative, future-oriented reinterpretation of the classic notebook or laptop computer. Elegant in appearance, the 13.3&quot; OLED screen system folds down to a practical size of 7.2&quot; so that it can be carried easily like a small paper book.
[1]Product Name: HP Pavilion 15 x360; Product Category: Laptop; Product Description: Versatile applications and careful design are the hallmarks of the HP Pavilion 15 x360. It can be used in tent mode for watching videos, in tablet mode as a digital drawing pad and as a laptop it is a reliable working tool.
[2]The IdeaPad U430s also offers users a pleasing tactile experience due to a special surface finish that makes it feel soft and natural to the touch. In a highly inspired manner, the design of the IdeaPad U430s places the user centre-stage – it has emerged as a laptop with fascinatingly new qualities.
[3]An Nvidia Tegra 4 processor provides the required speed for an optimised graphics presentation. The notebo

In [45]:
import re

def find_bracketed_one(text):
  """
  Checks if the text contains '[1]'.

  Args:
    text: The input string to search in.

  Returns:
    True if '[1]' is found, False otherwise.
  """
  if re.search(r"\[1\]", text):
    return True
  else:
    return False

# Example usage:
text1 = "This is a test string with [1] in it."
text2 = "This string does not have the sequence."
text3 = "This one has [2] but not [1]."
text4 = "This one has [1] in the beginning."

print(f"'{text1}': {find_bracketed_one(text1)}")
print(f"'{text2}': {find_bracketed_one(text2)}")
print(f"'{text3}': {find_bracketed_one(text3)}")

'This is a test string with [1] in it.': True
'This string does not have the sequence.': False
'This one has [2] but not [1].': True


In [47]:
re.search(r"\[1\]", text1)

re.Match